In [1]:
import sys
sys.path.append('..')
from src.includes import *

model_name = 'blstm'
data_set_name = 'Pidgin'
home_path = '/home/swj/VD/'
ways = 'AST'
tokenizer_saved_path = '/home/swj/VD/embedding/'

#change this
#all_data_path = home_path + 'data/Six_projects/'
all_data_path = home_path + 'data/ALL/'
data_path = all_data_path + data_set_name + '/'


model_path = home_path + 'trained_model/' + data_set_name + '/'
# repre_saved_path = '/home/swj/VD/Representation/PLSTM_myway/LibTIFF/'
# data_path_repre = '/home/swj/VD/data/Six_projects/LibTIFF/Non_vulnerable_functions/'
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# all_data_list, all_data_list_id = loadData(all_data_path)
# total_list, total_list_id = loadData(data_path)
all_data_list, all_data_list_id = loadAST(all_data_path)
total_list, total_list_id = loadAST(data_path)

verbose("Perform tokenization ....")
Tokenization(all_data_list)
verbose("Tokenization completed!")
verbose("-------------------------------------------------------")   
verbose("Perform code embedding ....")
embedding_model = Embedding_Model(config)
'''total_sequnces: variable lengths of number of samples vectors'''
'''word_index: a map of word to vectors'''
total_sequences, word_index = embedding_model.LoadTokenizer(total_list)
embedding_model.TrainWordToVec(total_list)
embedding_matrix, embedding_dim = embedding_model.ApplyWordToVec(word_index)            
verbose ("Word2vec loaded! ")
            
verbose("Pad the sequence to unified length...")
total_list_pad = padding(total_sequences)
verbose("Patition the data ....")
data_tuple = patitionData(total_list_pad, total_list_id)  
        
train_set_x = data_tuple[0]
train_set_y = np.asarray(data_tuple[1]).flatten()

test_set_x = data_tuple[2]
test_set_y = np.asarray(data_tuple[3]).flatten()

train_set = train_set_x, train_set_y
test_set = test_set_x, test_set_y
y_test = test_set_y

train_set_x = torch.Tensor(train_set_x).long().to(device) # transform to torch tensor
train_set_y = torch.Tensor(train_set_y).to(device)
train_dataset = TensorDataset(train_set_x,train_set_y) # create your datset
train_loader = DataLoader(train_dataset, batch_size=128) # create your dataloader

test_set_x = torch.Tensor(test_set_x).long().to(device) # transform to torch tensor
test_set_y = torch.Tensor(test_set_y).to(device)
test_dataset = TensorDataset(test_set_x,test_set_y) # create your datset
test_loader = DataLoader(test_dataset, batch_size=1) # create your dataloader

verbose ("Data Shapes Are:")
verbose ("Data processing completed!")
verbose ("-------------------------------------------------------")
verbose ("There are " + str(len(train_set_x)) + " total samples in the training set. " + str(torch.count_nonzero(train_set_y)) + " vulnerable samples. ")
verbose ("There are " + str(len(test_set_x)) + " total samples in the test set. " + str(torch.count_nonzero(test_set_y)) + " vulnerable samples. ")

weights=torch.from_numpy(embedding_matrix)
    

[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/ALL/....
[INFO] The length of the loaded data list is : 39940
[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/ALL/Pidgin/....
[INFO] The length of the loaded data list is : 8655
[INFO] Perform tokenization ....
[INFO] Tokenization completed!
[INFO] -------------------------------------------------------
[INFO] Perform code embedding ....
----------------------------------------
Start training the Word2Vec model. Please wait.. 
Model training completed!
----------------------------------------
The trained word2vec model: 
Word2Vec(vocab=750, vector_size=100, alpha=0.025)
-------------------------------------------------------
Loading trained Word2vec model. 
The trained word2vec model: 
<_io.TextIOWrapper name='/home/swj/VD/embedding/w2v_model.txt' mode='r' encoding='UTF-8'>
Found 751 word vectors.
[INFO] Word2vec loaded! 
[INFO] Pad the sequence to unified length...
[INFO] Patition the data ....
[INFO] Data Shapes 

In [2]:
recording_type = 'loss&acc'
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

#torch.cuda.set_per_process_memory_fraction(0.5, 0)
print(device)
    
    
bplstm = BPLSTMnet(weights).to(device)
blstm = BLSTMnet(weights).to(device)
criterion = nn.BCEWithLogitsLoss()
if model_name == 'blstm':
    optimizer = optim.Adam(blstm.parameters(), lr=0.001)
elif model_name == 'dlstm':
    optimizer = optim.Adam(bplstm.parameters(), lr=0.001)
epochs = 60

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    epoch_loss, epoch_acc = 0, 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        #print('data has shape: ' + str(data.shape))
        output = model(data)
        loss = criterion(output, target.unsqueeze(1))
        acc = binary_acc(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')
    return epoch_loss, epoch_acc


for epoch in range(epochs):
    time1 = time.time()
    if model_name == 'blstm':
        epoch_loss, epoch_acc = train(blstm, device, train_loader, optimizer, epoch)
    elif model_name == 'dlstm':
        epoch_loss, epoch_acc = train(bplstm, device, train_loader, optimizer, epoch)
    time2 = time.time()
    print(f'Epoch {epoch} elapse {time2 - time1}s')
    
if not os.path.exists(model_path): os.makedirs(model_path)
if model_name == 'blstm':
    torch.save(blstm.state_dict(),  model_path + model_name + '_' + ways + '_model.h5')
elif model_name == 'dlstm':
    torch.save(bplstm.state_dict(), model_path + model_name + '_' + ways + '_model.h5')


cuda
Epoch 000: | Loss: 0.01146 | Acc: 1.818
Epoch 000: | Loss: 0.01146 | Acc: 3.636
Epoch 000: | Loss: 0.01592 | Acc: 5.436
Epoch 000: | Loss: 0.01592 | Acc: 7.255
Epoch 000: | Loss: 0.01592 | Acc: 9.073
Epoch 000: | Loss: 0.01592 | Acc: 10.891
Epoch 000: | Loss: 0.01592 | Acc: 12.709
Epoch 000: | Loss: 0.02177 | Acc: 14.509
Epoch 000: | Loss: 0.02177 | Acc: 16.327
Epoch 000: | Loss: 0.02177 | Acc: 18.145
Epoch 000: | Loss: 0.02177 | Acc: 19.964
Epoch 000: | Loss: 0.02177 | Acc: 21.782
Epoch 000: | Loss: 0.02177 | Acc: 23.600
Epoch 000: | Loss: 0.02177 | Acc: 25.418
Epoch 000: | Loss: 0.02177 | Acc: 27.236
Epoch 000: | Loss: 0.02551 | Acc: 29.036
Epoch 000: | Loss: 0.02551 | Acc: 30.855
Epoch 000: | Loss: 0.02551 | Acc: 32.673
Epoch 000: | Loss: 0.02551 | Acc: 34.491
Epoch 000: | Loss: 0.02551 | Acc: 36.309
Epoch 000: | Loss: 0.02554 | Acc: 38.127
Epoch 000: | Loss: 0.02560 | Acc: 39.945
Epoch 000: | Loss: 0.02645 | Acc: 41.745
Epoch 000: | Loss: 0.02665 | Acc: 43.564
Epoch 000: | Los

In [2]:
import sys
sys.path.append('..')

from src.includes import *


home_path = '/home/swj/VD/'
model_name = 'dlstm'
data_set_name = 'VLC'
ways = 'AST'
code_type = 'C++'
vulnerability = 'vul'
repre_saved_path = home_path + 'Representation/' + model_name + '_' + ways + '/' + code_type + '/' + data_set_name + '/'
model_path = '/home/swj/VD/trained_model/' + data_set_name + '/' + model_name + '_' + ways + '_model.h5'

tokenizer_saved_path = '/home/swj/VD/embedding/'
#change this
#all_data_path = home_path + 'data/Six_projects/'
all_data_path = home_path + 'data/ALL/'
data_path = all_data_path + data_set_name + '/'



# all_data_list, all_data_list_id = loadData(all_data_path)
# total_list, total_list_id = loadData(data_path)
all_data_list, all_data_list_id = loadAST(all_data_path)
total_list, total_list_id = loadAST(data_path)

verbose("Perform tokenization ....")
Tokenization(all_data_list)
verbose("Tokenization completed!")
verbose("-------------------------------------------------------")   
verbose("Perform code embedding ....")
embedding_model = Embedding_Model(config)
'''total_sequnces: variable lengths of number of samples vectors'''
'''word_index: a map of word to vectors'''
total_sequences, word_index = embedding_model.LoadTokenizer(total_list)
embedding_model.TrainWordToVec(total_list)
embedding_matrix, embedding_dim = embedding_model.ApplyWordToVec(word_index)
weights=torch.from_numpy(embedding_matrix)

#obtain representations
def ObtainRepresentations_by_batch_size(input_sequences, feature_model, BATCH_SIZE):
    num_batches_per_epoch = int((len(input_sequences) - 1) / BATCH_SIZE) + 1
    data_size = len(input_sequences)
    representations_total = []
    model = feature_model
    for batch_num in range(num_batches_per_epoch):
        start_index = batch_num * BATCH_SIZE
        end_index = min((batch_num + 1) * BATCH_SIZE, data_size)
        print ("-------start_index------------")
        print (start_index)
        print ("-------end_index------------")
        print (end_index)
        print(input_sequences[start_index: end_index].shape)
        representations = model(input_sequences[start_index: end_index])
        representations_total = representations_total + representations.tolist()
    return np.asarray(representations_total)


if model_name == 'blstm':
    net = BLSTMnet(weights).to(device)
elif model_name == 'dlstm':
    net = BPLSTMnet(weights).to(device)
elif model_name == 'lstm':
    net = LSTMnet(weights).to(device)
    
#change this
net.load_state_dict(torch.load(model_path))
net.eval()

feature_model = None
if model_name == 'blstm':
    feature_model = BLSTM_Feature_Net(net, weights).to(device)
elif model_name == 'dlstm':
    feature_model = BPLSTM_Feature_Net(net, weights).to(device)

feature_model.eval()

if vulnerability == 'vul':
    data_list, data_list_id = loadAST(data_path + 'Vulnerable_functions/')
else: 
    data_list, data_list_id = loadAST(data_path + 'Non_vulnerable_functions/')
tokenizer = LoadToknizer(tokenizer_saved_path + 'tokenizer.pickle')

data_sequence = tokenizer.texts_to_sequences(data_list)
data_list_pad = padding(data_sequence)
data_list_pad = torch.Tensor(data_list_pad).long().to(device)
start_time = time.time()
obtained_repre = ObtainRepresentations_by_batch_size(data_list_pad, feature_model, 128)
end_time = time.time()
print("Saving the obtained representations....") 
print('uisng' + str(end_time - start_time) + 's')
if not os.path.exists(repre_saved_path): os.makedirs(repre_saved_path)

if vulnerability == 'vul':
    SavedPickle(repre_saved_path + "Vul.pkl", obtained_repre)
else:
    SavedPickle(repre_saved_path + "NonVul.pkl", obtained_repre)

print("The obtained representations are saved in: " + str(repre_saved_path) + ".")

[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/ALL/....
[INFO] The length of the loaded data list is : 39940
[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/ALL/VLC/....
[INFO] The length of the loaded data list is : 6159
[INFO] Perform tokenization ....
[INFO] Tokenization completed!
[INFO] -------------------------------------------------------
[INFO] Perform code embedding ....
----------------------------------------
Start training the Word2Vec model. Please wait.. 
Model training completed!
----------------------------------------
The trained word2vec model: 
Word2Vec(vocab=623, vector_size=100, alpha=0.025)
-------------------------------------------------------
Loading trained Word2vec model. 
The trained word2vec model: 
<_io.TextIOWrapper name='/home/swj/VD/embedding/w2v_model.txt' mode='r' encoding='UTF-8'>
Found 624 word vectors.
[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/ALL/VLC/Vulnerable_functions/....
[INFO] The length of the loa